<h1> Imports <h1>

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

#defining dates
import datetime as dt
import time

#download stock data
import yfinance as yf

#file handling
import os
from os import listdir
from os.path import isfile, join

#additional plotting tools
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

#make plotly work in Jupyter notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

from plotly.subplots import make_subplots

import warnings
warnings.simplefilter("ignore")


<h1> Constants <h1>

In [34]:
PATH = "./stock_data/"

#Start end date constants
S_DATE = "2017-04-16"
E_DATE = "2022-04-16"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)

<h1> Get Dataframe from CSV <h1>

In [35]:
def get_stock_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col = 0)
    except FileNotFoundError:
        print("File Doesn't exist")
    else:
        return df


<h1> Get all Stocks Downloaded in List <h1>

In [36]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
#tickers

tickers.sort()
len(tickers)


795

<h1> Add Daily Returns <h1>

In [37]:
#caluclate price changes for everyday
def add_daily_return_to_df(df):
    #add new column to df
    df['daily_return'] = (df['Close'] / df['Close'].shift(1)) -1
    #df.to_csv(PATH + ticker + '.csv')
    return df

<h1> Add Cumulative Returns <h1>

In [38]:
#calculate cumulative price changes
def add_cum_return_to_df(df):
    #add new column to df
    df['cum_return'] = (1+df['daily_return']).cumprod()
    #df.to_csv(PATH + ticker + '.csv')
    return df

# Add Bollinger Bands 

<p> A Bollinger Band is a technical analysis tool defined by a set of trendlines plotted two standard deviations away form a simple moving average of a security's price. There are three lines that compose the Bollinger Bands: A SMA (middle) and an upper and lower band. The upper and lower bands are typically 2 standard deviations from a 20-day SMA. <p>

<p> The closer the price moves to the upper band, the more overbought the market is, and the closer the price moves toward the lower band, the more oversold the market. The tightening and broadening of the bands suggest a measure of stability and more volatility respectively. A strong trend or break is suggested if the prices moves outside of the bands. If the bands squeeze the SMA, it is believed a volatile move will be made soon <P>

In [39]:
def add_bollinger_bands(df):
    df['moving_average'] = df['Close'].rolling(window=20).mean()
    #Utilizing 1.96 because that is two standard deviations away
    df['upper_band'] = df['moving_average'] + 1.96 * df['Close'].rolling(window=20).std()
    df['lower_band'] = df['moving_average'] - 1.96 * df['Close'].rolling(window=20).std()
    #df.to_csv(PATH + ticker + '.csv')
    return df

<h1> Add Ichimoku Data to Dataframe <h1>

The Ichimoku (One Look) is considered an all in one indicator that shows support and resistance levels, as well as momentum and trend direction. It computes a "cloud" that attempts to forecast where the price may find support or resistance. The Ichimoku Cloud comprises of 5 lines:
* Conversion Line (Tenkan-sen): Nine-period average - Represents support, resistance, and reversals. Used for short term action
* Baseline (Kijun-sen): 26-period average - Represents support, resistance, and confirms trend changes. Used to evaluate medium term trends. Called the baseline because it lags the price.
* Leading Span A (Senkou A): Average of the Conversion and Base Value over 26 period - Identify future areas of support and resistance
* Leading Span B (Senkou B): Average of Conversion and Base Line over 52 period - Identify future support and resistance
* Lagging Span (Chikou): Lagging closing price - Price shifted back 26 periods. Shows possible support and resistance
* Cloud (Kumo): Space between Span A and B. Represents the divergence in price evolution

When price is above the cloud, the trend is bullish and vice versa. The trend is strengthened if the cloud is moving in the same direction as the price. 

In [40]:
def add_Ichimoku(df):
    #Conversion Line (Highest Value in period + Lowest)/2 in 9 sessions
    high_value = df['High'].rolling(window=9).max()
    low_value = df['Low'].rolling(window=9).min()
    df['Conversion'] = (high_value + low_value)/2

    #Base Line (Highest Value in Period + Lowest)/2 over 26 sessions
    base_high = df['High'].rolling(window=26).max()
    base_low = df['Low'].rolling(window=26).min()
    df['Baseline'] = (base_high + base_low)/2

    #Span A (Conversion Value + Base Value)/2 over 26 sessions
    df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2).shift(26)

    #Span B (Conversion Value + Base Value)/2 over 52 sessions
    spanB_high = df['High'].rolling(window=52).max()
    spanB_low = df['Low'].rolling(window=52).min()
    df['SpanB'] = ((spanB_high + spanB_low) / 2).shift(26)

    #Lagging Span (Price shifted back 26 periods)
    df['Lagging'] = df['Close'].shift(-26)
    return df

# Add Daily, Cumulative, Bollinger Bands, Ichimoku to Dataframes

In [41]:
# Testing with one ticker
# writes calculated Ichimoku values into csv files for each security
try:
    print("Working on :", "SPY")
    new_df = get_stock_df_from_csv("SPY")
    new_df = add_daily_return_to_df(new_df)
    new_df = add_cum_return_to_df(new_df)
    new_df = add_bollinger_bands(new_df)
    new_df = add_Ichimoku(new_df)
    new_df.to_csv(PATH + 'SPY' + '.csv')
except Exception as ex:
    print("ex")
    

Working on : SPY


In [42]:
# for x in tickers:
#     try:
#         print("Working on :", x)
#         new_df = get_stock_df_from_csv(x)
#         new_df = add_daily_return_to_df(new_df)
#         new_df = add_cum_return_to_df(new_df)
#         new_df = add_bollinger_bands(new_df)
#         new_df = add_Ichimoku(new_df)
#         new_df.to_csv(PATH + x + '.csv')
#     except Exception as ex:
#         print("ex")